## Homework 6 - Question 1
Allison Aprile

April 7, 2024

In [1]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az

Based on the following examples:                   
* https://static.us.edusercontent.com/files/k20FshFhGbGWEkUev2BZcfPu
* https://areding.github.io/6420-pymc/unit8/Unit8-tte-gastric.html

### <mark>Answer</mark>
* **Uncensored** 95% HPD Credible Set for $\beta_1$: [-0.175, 0.773]
* **Censored** 95% HPD Credible Set for $\beta_1$: [0.023, 1.157]

### Data Setup

In [2]:
# Read data
data = pd.read_csv('data/tongue.csv')

# Update column names
data = data.rename(columns={
                           '1': 'Tumor Profile', 
                           '1.1': 'Weeks', 
                           '0': 'Censored'
                          }, )

# Create censored value column
## Per homework instructions, 0=censored and 1=observed
def censor_values(i):
    # If observed, use 'Weeks' value
    if i['Censored'] == 1:
        return i['Weeks']
    # If censored, use infinity
    else:
        return np.inf
        
data['Censored Values'] = data.apply(censor_values, axis=1)

data

,Tumor Profile,Weeks,Censored,Censored Values
0,1,3,0,inf
1,1,3,0,inf
2,1,4,0,inf
3,1,10,0,inf
4,1,13,0,inf
...,...,...,...,...
74,2,67,1,67.0
75,2,76,1,76.0
76,2,104,1,104.0
77,2,176,1,176.0


In [3]:
# Unpack into arrays
tumor = data['Tumor Profile'].to_numpy()
weeks = data['Weeks'].to_numpy()
censored = data['Censored Values'].to_numpy()

### Model Definition

We can use the Weinbull Distribution to model this time-to-death, $T$, data. It is parameterized in PyMC as:


$T \sim Weinbull(\alpha, \beta)$

$\quad \alpha = \gamma > 0, \ \ \beta = \lambda^{-1/\alpha} > 0$

**CDF:** $F(t | \alpha, \beta) = 1 - e^{-(t|\beta)^\alpha}$, for $t > 0$

We set a non-informative Exponential prior on $\alpha$, as this is required to be positive for the Weinbull distribution. For the intercept and slope ($\beta_0$ and $\beta_1$, respectively), we select more non-informative Normal priors, centered at 0 and with a small precision ($\tau = 0.01$). We make these slightly more informative (but still non-informative) for the censored model ($\tau = 0.0001$).

#### Uncensored Model
Using 'Weeks' values directly, i.e. taking the censored data at face value.

In [4]:
with pm.Model() as uncensored_model:
    # Non-informative priors
    alpha = pm.Exponential('alpha', 0.5)
    beta0 = pm.Normal('beta0', 0, 10)
    beta1 = pm.Normal('beta1', 0, 10)

    # Beta formulation (with natural logarithm link function)
    lam = pm.math.exp(beta0 + beta1 * tumor)
    beta = lam ** (-1 / alpha)

    # Likelihood
    pm.Weibull('likelihood', alpha=alpha, beta=beta, observed=weeks)

    # Median (less affected by outliers)
    median1 = pm.Deterministic(
        'median1',
        (np.log(2) * (pm.math.exp(beta0 + beta1 * 1)) ** (-1 / alpha)),
    )
    median2 = pm.Deterministic(
        'median2',
        (np.log(2) * (pm.math.exp(beta0 + beta1 * 2)) ** (-1 / alpha)),
    )

    # Sample from posterior
    uncensored_trace = pm.sample(3000, init='jitter+adapt_diag_grad', target_accept=0.9)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta0, beta1]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 5 seconds.


In [5]:
# Statistics and diagnostics summary
az.summary(uncensored_trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-5.159,0.584,-6.297,-4.033,0.010,0.007,3112.0,4174.0,1.0
beta1,0.307,0.240,-0.175,0.773,0.004,0.003,4515.0,4591.0,1.0
alpha,1.091,0.096,0.906,1.283,0.002,0.001,3442.0,4132.0,1.0
median1,59.647,8.010,44.320,75.373,0.103,0.073,5969.0,7314.0,1.0
median2,45.333,8.513,29.846,62.077,0.104,0.075,6889.0,7496.0,1.0


Note: Per Aaron's suggestion in the [March 21st office hours](https://gatech.instructure.com/courses/355300/external_tools/17731), we can be fairly confident that we have good sampling coverage as $\hat{r} < 1.01$.

<mark>The 95% HPD Credible Set for the slope $\beta_1$ is [-0.175, 0.773].</mark>

#### Censored Model
Using 'Weeks' values, but counting for the censoring indicator.

In [6]:
with pm.Model() as censored_model:
    # Non-informative priors
    alpha = pm.Exponential('alpha', 0.5)
    beta0 = pm.Normal('beta0', 0, 100)
    beta1 = pm.Normal('beta1', 0, 100)

    # Beta formulation (with natural logarithm link function)
    lam = pm.math.exp(beta0 + beta1 * tumor)
    beta = lam ** (-1 / alpha)

    # Likelihood (accounting for right-censored data)
    obs_latent = pm.Weibull.dist(alpha=alpha, beta=beta)
    pm.Censored('likelihood', obs_latent, lower=None, upper=censored, observed=weeks)

    # Median (less affected by outliers)
    median_1 = pm.Deterministic(
        'aneuploid_median',
        (np.log(2) * (pm.math.exp(beta0 + beta1 * 1)) ** (-1 / alpha)),
    )
    median_2 = pm.Deterministic(
        'diploid_median',
        (np.log(2) * (pm.math.exp(beta0 + beta1 * 2)) ** (-1 / alpha)),
    )

    # Sample from posterior
    censored_trace = pm.sample(3000, init='jitter+adapt_diag_grad', target_accept=0.9)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta0, beta1]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 5 seconds.


In [7]:
# Statistics and diagnostic summary
az.summary(censored_trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-4.787,0.639,-6.030,-3.532,0.011,0.008,3297.0,3803.0,1.0
beta1,0.569,0.287,0.023,1.157,0.004,0.003,4207.0,4979.0,1.0
alpha,0.841,0.097,0.657,1.036,0.001,0.001,4265.0,5187.0,1.0
aneuploid_median,108.130,26.034,62.670,158.783,0.306,0.217,7302.0,7957.0,1.0
diploid_median,55.019,15.147,28.837,84.239,0.182,0.129,6899.0,7709.0,1.0


Note: As with the Uncensored Model, we can be fairly confident that we have good sampling coverage as $\hat{r} < 1.01$.

<mark>The 95% HPD Credible Set for the slope $\beta_1$ is [0.023, 1.157].</mark>